<a href="https://colab.research.google.com/github/newmantic/GMM/blob/main/GMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.stats import multivariate_normal

class GaussianMixtureModel:
    def __init__(self, n_components, max_iter=100, tol=1e-6):
        """
        Initialize the GMM with the number of components and other parameters.

        Parameters:
        n_components (int): Number of Gaussian components in the mixture.
        max_iter (int): Maximum number of iterations for the EM algorithm.
        tol (float): Convergence threshold for the log-likelihood.
        """
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol

    def fit(self, X):
        """
        Fit the GMM to the data using the EM algorithm.

        Parameters:
        X (np.ndarray): Data matrix of shape (n_samples, n_features).
        """
        n_samples, n_features = X.shape

        # Initialize the parameters
        self.weights = np.full(self.n_components, 1 / self.n_components)
        self.means = np.random.rand(self.n_components, n_features)
        self.covariances = np.array([np.eye(n_features)] * self.n_components)
        log_likelihood_prev = None

        for iteration in range(self.max_iter):
            # E-step: Compute responsibilities
            responsibilities = np.zeros((n_samples, self.n_components))
            for k in range(self.n_components):
                responsibilities[:, k] = self.weights[k] * multivariate_normal(self.means[k], self.covariances[k]).pdf(X)
            responsibilities /= responsibilities.sum(axis=1, keepdims=True)

            # M-step: Update the parameters
            Nk = responsibilities.sum(axis=0)
            self.weights = Nk / n_samples
            self.means = np.dot(responsibilities.T, X) / Nk[:, np.newaxis]
            for k in range(self.n_components):
                X_centered = X - self.means[k]
                self.covariances[k] = np.dot(responsibilities[:, k] * X_centered.T, X_centered) / Nk[k]

            # Compute the log-likelihood
            log_likelihood = np.sum(np.log(np.sum([
                self.weights[k] * multivariate_normal(self.means[k], self.covariances[k]).pdf(X)
                for k in range(self.n_components)
            ], axis=0)))

            # Check for convergence
            if log_likelihood_prev is not None and abs(log_likelihood - log_likelihood_prev) < self.tol:
                break
            log_likelihood_prev = log_likelihood

    def predict_proba(self, X):
        """
        Predict the probabilities of each sample in X belonging to each component.

        Parameters:
        X (np.ndarray): Data matrix of shape (n_samples, n_features).

        Returns:
        np.ndarray: Probability matrix of shape (n_samples, n_components).
        """
        n_samples = X.shape[0]
        probabilities = np.zeros((n_samples, self.n_components))
        for k in range(self.n_components):
            probabilities[:, k] = self.weights[k] * multivariate_normal(self.means[k], self.covariances[k]).pdf(X)
        probabilities /= probabilities.sum(axis=1, keepdims=True)
        return probabilities

    def sample(self, n_samples):
        """
        Sample new data points from the fitted GMM.

        Parameters:
        n_samples (int): Number of samples to generate.

        Returns:
        np.ndarray: Generated data of shape (n_samples, n_features).
        """
        samples = []
        component_choices = np.random.choice(self.n_components, size=n_samples, p=self.weights)
        for k in component_choices:
            samples.append(np.random.multivariate_normal(self.means[k], self.covariances[k]))
        return np.array(samples)

# Testable example:
# Generate some synthetic data
np.random.seed(42)
X = np.vstack([
    np.random.multivariate_normal([0, 0], [[1, 0.5], [0.5, 1]], 100),
    np.random.multivariate_normal([5, 5], [[1, -0.5], [-0.5, 1]], 100)
])

# Fit the GMM
gmm = GaussianMixtureModel(n_components=2)
gmm.fit(X)

# Predict probabilities for the data points
probabilities = gmm.predict_proba(X)
print(f"Predicted probabilities:\n{probabilities[:5]}")

# Sample new data points from the GMM
samples = gmm.sample(5)
print(f"Generated samples:\n{samples}")

Predicted probabilities:
[[1.00000000e+00 8.27754248e-31]
 [1.00000000e+00 1.90789939e-31]
 [1.00000000e+00 2.36820330e-24]
 [1.00000000e+00 5.73742631e-41]
 [1.00000000e+00 3.67151367e-22]]
Generated samples:
[[4.51245428 4.08656052]
 [6.44138583 4.24201068]
 [4.57495362 6.78040247]
 [5.56375616 4.28671721]
 [5.57079248 6.3388876 ]]
